In [1]:
import tensorflow as tf

In [2]:
graph = tf.Graph()

In [3]:
with graph.as_default():
    
    with tf.name_scope("variables"):
        # Variable to keep track of how many times the graph has been run
        global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name="global_step")
        # Variable that keeps track of the sum of all output values over time:
        total_output = tf.Variable(0.0, dtype=tf.float32, trainable=False, name="total_output")
        
    with tf.name_scope("transformation"):
        # Separate input layer
        with tf.name_scope("input"):
            # Create input placeholder- takes in a Vector
            a = tf.placeholder(tf.float32, shape=[None], name="input_placeholder_a")
            
        # Separate middle layer
        with tf.name_scope("intermediate_layer"):
            b = tf.reduce_prod(a, name="product_b")
            c = tf.reduce_sum(a, name="sum_c")
            
        # Separate output layer
        with tf.name_scope("output"):
            output = tf.add(b, c, name="output")
            
    with tf.name_scope("update"):
        # Increments the total_output Variable by the latest output
        update_total = total_output.assign_add(output)
        # Increments the above `global_step` Variable, should be run whenever the graph is run
        increment_step = global_step.assign_add(1)
    
    with tf.name_scope("summaries"):
        avg = tf.div(update_total, tf.cast(increment_step, tf.float32), name="average")
        # Creates summaries for output node
        tf.summary.scalar("output_summary", output)
        tf.summary.scalar("total_summary", update_total)
        tf.summary.scalar("average_summary", avg)
        
    with tf.name_scope("global_ops"):
        # Initialization Op
        init = tf.initialize_all_variables()
        # Merge all summaries into one Operation
        merged_summaries = tf.summary.merge_all()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [4]:
sess = tf.Session(graph=graph)
writer = tf.summary.FileWriter('./improved_graph', graph)

In [5]:
sess.run(init)

In [6]:
def run_graph(input_tensor):
    feed_dict = {a: input_tensor}
    _, step, summary = sess.run([output, increment_step, merged_summaries], feed_dict=feed_dict)
    writer.add_summary(summary, global_step=step)

In [7]:
run_graph([2,8])

In [8]:
run_graph([3,1,3,3])

In [9]:
run_graph([8])

In [10]:
run_graph([1,2,3])

In [11]:
run_graph([11,4])

In [12]:
run_graph([4,1])

In [13]:
run_graph([7,3,1])

In [14]:
run_graph([6,3])

In [15]:
run_graph([0,2])

In [16]:
writer.flush()

In [17]:
writer.close()
sess.close()